In [837]:
# Import base dependencies
import numpy as np
import pandas as pd
from pathlib import Path
import yfinance as yf
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [838]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"AGX", 'basis':113.92}, 
             {'symbol':"APP", 'basis':257.00}, {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"ATGE"}, {'symbol':"BRK-B"}, 
             {'symbol':"CCL", 'basis':18.30}, {'symbol':"CLS", 'basis':74.04}, 
             {'symbol':"CRDO", 'basis': 38.36}, {'symbol':"EAT", 'basis':135.80}, 
             {'symbol':"EZPW"}, {'symbol':"MFC"}, 
             {'symbol':"PPC"}, {'symbol':"STRL"}, {'symbol':"SKYW", 'basis':84.47}, 
             {'symbol':"UBER", 'basis':61.47}, {'symbol':"WFC", 'basis':67.06}
             ]

# Brokerage
brokerage_funds = [{'symbol':"AGX", 'basis':117.784}, {'symbol':"APP", 'basis':262.705}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"ATGE", 'basis':97.431}, {'symbol':"BAI", 'basis':25.059}, 
                   {'symbol':"CCL", 'basis':20.829}, {'symbol':"CLS", 'basis':82.22}, 
                   {'symbol':"CRDO", 'basis':41.848}, {'symbol':"FBTC", 'basis':77.905}, 
                   {'symbol':"NVDA", 'basis':99.373}, {'symbol':"QTUM", 'basis':82.675}, 
                   {'symbol':"SKYW", 'basis':85.885}, 
                   {'symbol':"VOOG", 'basis':0}, {'symbol':"WFC", 'basis':73.186}
                   ]

In [ ]:
# get historical market data
def get_history(stock_symbol):
    # Establish ticker information
    stock_research = yf.Ticker(stock_symbol)
    history = stock_research.history(period="max")

    # Filter if desired
    filter = False
    filter_date = '2025-04-01'
    if filter:
        history = history.loc[:filter_date]
        return history
    
    # Return stock price/volume history
    return history

def calculate_vwap(df):
    # Trim raw history for calculation
    vwap_analysis = df[-63:].copy()

    # Calculate VWAP and add to df
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], ndigits=2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], ndigits=2)
    
    return vwap_analysis[-1:].copy()

def calculate_quartiles(df):
    # Trim df for quartile analysis
    quartiles = df[-21:].copy()

    # Calculate upper and lower quartiles
    q1 = round(np.quantile(quartiles['Low'], 0.25), ndigits=2)
    q3 = round(np.quantile(quartiles['High'], 0.75), ndigits=2)

    # Return final df
    return q1, q3

def build_analysis_table(ticker_symbols):
    # Establish list for exporting
    portfolio = []

    # Iterate through tickers to retrieve historical data and perform calculations
    for ticker in ticker_symbols:
        # Get data
        raw_data = get_history(ticker['symbol'])
        vwap_data = calculate_vwap(raw_data)
        q1, q3 = calculate_quartiles(raw_data)

        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Close'].iloc[-1], ndigits=2)
        entry_price = round(vwap_data['Entry'].iloc[0], ndigits=2)
        exit_price = round(vwap_data['Exit'].iloc[0], ndigits=2)

        # Margin of Safety: 20% below VWAP Entry point
        buy_threshold = round(entry_price * 0.95, ndigits=2)

        # Volume filter: 50% of 20-day avg
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.3

        # Decision (for display or later use)
        decision = 'Hold'
        if market_price <= min(buy_threshold, q1) and volume_ok:
            decision = "Buy"
        elif market_price >= max(exit_price, q3) and volume_ok:
            decision = "Sell"
        
        portfolio.append([ticker['symbol'], basis, market_price, entry_price, q1, buy_threshold, exit_price, q3, decision])

    # Return list element for ticker symbol
    return portfolio

In [840]:
b_portfolio = build_analysis_table(brokerage_funds)
brokerage_df = pd.DataFrame(b_portfolio, 
                            columns=['ticker', 'basis', 'price', 'entry', 'q1', 'buy', 'exit', 'q3', 'rating'])

In [841]:
brokerage_df

,ticker,basis,price,entry,q1,buy,exit,q3,rating
0,AGX,117.784,147.04,131.93,126.68,125.33,143.28,149.45,Hold
1,APP,262.705,254.68,235.87,226.10,224.08,264.74,278.88,Hold
2,ARQT,15.996,14.46,13.37,12.79,12.70,14.83,15.02,Hold
3,ATGE,97.431,105.96,99.86,98.40,94.87,105.51,107.05,Hold
4,BAI,25.059,21.99,19.88,20.15,18.89,21.11,22.18,Hold
5,CCL,20.829,18.71,17.26,17.08,16.40,18.61,19.33,Hold
6,CLS,82.220,83.09,75.28,75.15,71.52,83.01,86.10,Hold
7,CRDO,41.848,41.04,36.19,35.58,34.38,40.32,43.66,Hold
8,FBTC,77.905,82.38,72.21,70.93,68.60,75.10,81.83,Hold
9,NVDA,99.373,104.86,99.55,97.53,94.57,107.38,111.55,Hold


In [842]:
r_portfolio = build_analysis_table(ira_funds)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 'entry', 'q1', 'buy', 'exit', 'q3', 'rating'])

In [843]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,entry,q1,buy,exit,q3,rating
0,AGX,113.920,147.04,131.93,126.68,125.33,143.28,149.45,Hold
1,APP,257.000,255.02,235.87,226.10,224.08,264.74,278.88,Hold
2,ARQT,15.932,14.46,13.37,12.79,12.70,14.83,15.02,Hold
5,CCL,18.300,17.75,17.19,17.08,16.33,18.56,19.22,Hold
6,CLS,74.040,83.21,75.29,75.15,71.53,83.01,86.10,Hold
7,CRDO,38.360,41.04,36.19,35.58,34.38,40.32,43.66,Hold
8,EAT,135.800,135.12,140.07,133.60,133.07,151.26,155.71,Hold
13,SKYW,84.470,87.00,82.54,83.15,78.41,87.69,89.45,Hold
14,UBER,61.470,77.99,69.84,69.33,66.35,73.83,77.58,Hold
15,WFC,67.060,69.53,63.26,61.49,60.10,66.64,69.85,Hold


In [844]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,entry,q1,buy,exit,q3,rating
3,ATGE,0.0,105.96,99.86,98.40,94.87,105.51,107.05,Hold
4,BRK-B,0.0,527.95,504.22,498.61,479.01,525.31,535.92,Hold
9,EZPW,0.0,15.91,15.47,15.14,14.70,16.14,16.25,Hold
10,MFC,0.0,30.26,28.11,27.50,26.70,29.47,30.46,Hold
11,PPC,0.0,53.99,50.48,48.58,47.96,52.44,54.30,Hold
12,STRL,0.0,144.49,126.81,112.16,120.47,136.22,147.14,Hold


In [845]:
# Research new investments
if 1==1:
    custom_watchlist = [{'symbol':'EZPW'}, 
                        {'symbol':'QTWO'}, 
                        {'symbol':'PPC'}, 
                        {'symbol':'ATGE'}, 
                        {'symbol':'BRK-B'}, 
                        {'symbol':'TWLO'}, 
                        {'symbol':'MFC'}, 
                        {'symbol':'STRL'}, 
                        {'symbol':'GRBK'}, 
                        {'symbol':'UBER'}, 
                        {'symbol':'BLBD'}, 
                        {'symbol':'CAAP'}, 
                        {'symbol':'AMZN'}, 
                        {'symbol':'GOOG'}, 
                        {'symbol':'GOOGL'}, 
                        {'symbol':'GME'}, 
                        ] 
    watch_list = build_analysis_table(custom_watchlist)
else:
    symbol_input = input("Input symbol to research: ").upper()
    one_off_lookup = [{'symbol': symbol_input}]
    watch_list = build_analysis_table(one_off_lookup)

# Create watch list data frame
watch_list_df = pd.DataFrame(watch_list, 
                             columns=['ticker', 'basis', 'price', 'entry', 'q1', 'buy', 'q3', 'exit', 'rating'])

In [846]:
watch_list_df[watch_list_df['rating']=='Buy']

,ticker,basis,price,entry,q1,buy,q3,exit,rating


In [847]:
watch_list_df

,ticker,basis,price,entry,q1,buy,q3,exit,rating
0,EZPW,0,15.91,15.47,15.14,14.70,16.14,16.25,Hold
1,QTWO,0,78.07,72.56,71.35,68.93,76.86,79.28,Hold
2,PPC,0,53.99,50.48,48.58,47.96,52.44,54.30,Hold
3,ATGE,0,105.49,99.86,98.40,94.87,105.51,107.05,Hold
4,BRK-B,0,528.02,504.22,498.61,479.01,525.31,535.92,Hold
5,TWLO,0,93.00,86.05,82.11,81.75,91.59,95.28,Hold
6,MFC,0,30.26,28.11,27.50,26.70,29.47,30.46,Hold
7,STRL,0,144.49,126.81,112.16,120.47,136.22,147.14,Hold
8,GRBK,0,57.11,54.66,53.84,51.93,57.75,58.42,Hold
9,UBER,0,77.99,69.84,69.33,66.35,73.83,77.58,Hold


In [848]:
# Machine Learning

In [849]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [850]:
# Feature selection and normalization

In [851]:
# Build model

In [852]:
# Train model

In [853]:
# Evaluate model

In [854]:
# Refine model through hyperparameter tuning